In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
# Load FC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
no_snps_subs = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/fc_subs_no_snps.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []
subids = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    if sub in no_snps_subs:
        continue
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1155, 34716), (1155, 34716), (1155, 34716), (1155,), (1155,), (1155,)]


In [3]:
# Load big SNPs

snps = []
no_snps_subs = []

snps_file = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/snps_all_subs_big.pkl', 'rb'))

for sub in subids:
    if sub not in snps_file:
        print(sub)
        # no_snps_subs.append(sub)
    snps.append(snps_file[sub])

snps = np.stack(snps)
snps[np.isnan(snps)] = 0

print(snps.shape)

(1155, 35621)


In [4]:
import torch
from sklearn.linear_model import LogisticRegression

x = np.concatenate([snps == 0, snps == 1, snps == 2], axis=1)
reg_sex = LogisticRegression(C=100).fit(x, sex)
sex_w = torch.from_numpy(reg_sex.coef_[0]).float().cuda()
sex_i = reg_sex.intercept_[0]

reg_race = LogisticRegression(C=100).fit(x, race)
race_w = torch.from_numpy(reg_race.coef_[0]).float().cuda()
race_i = reg_race.intercept_[0]

print(sex_i, race_i)

-0.00030872895499649397 -2.4671035241755628e-05


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

class SNPVAE(nn.Module):
    def __init__(self, snpd, ld):
        super(SNPVAE, self).__init__()
        self.ld = ld
        self.enc1 = nn.Linear(snpd*3, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+2, 1000).float().cuda()
        self.dec20 = nn.Linear(1000, snpd).float().cuda()
        self.dec21 = nn.Linear(1000, snpd).float().cuda()
        self.dec22 = nn.Linear(1000, snpd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()#/(10**0.5)

    def dec(self, z, sex, race):
        z = torch.cat([z, sex.unsqueeze(1), race.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x0 = self.dec20(x)
        x1 = self.dec21(x)
        x2 = self.dec22(x)
        return torch.stack([x0, x1, x2], dim=1)

def latent_loss(z):
    C = z.T@z
    mu = torch.mean(z, dim=0)
    tgt1 = torch.eye(z.shape[-1]).float().cuda()*len(z)#/10
    tgt2 = torch.zeros(z.shape[-1]).float().cuda()
    lossC = rmse(C, tgt1)
    lossmu = rmse(mu, tgt2)
    return lossC, lossmu, C, mu

def decor_loss(z, sex, race):
    sex = sex - torch.mean(sex)
    race = race - torch.mean(race)
    sexp = torch.einsum('n,nz->z', sex, z)
    racep = torch.einsum('n,nz->z', race, z)
    tgt = torch.zeros(z.shape[-1]).float().cuda()
    loss_sex = rmse(sexp, tgt)
    loss_race = rmse(racep, tgt)
    return loss_sex + loss_race

def klloss(z):
    N = z.shape[1]
    sigma = torch.std(z, dim=0)
    mu = torch.mean(z, dim=0)
    loss = torch.sum(sigma**2)+torch.sum(mu**2)-2*torch.sum(torch.log(sigma))
    return loss

def klloss2(z):
    sigma = torch.einsum('na,nb->nab', z, z)
    tr = sigma.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1)
    ld = torch.logdet(sigma)
    print(ld)
    mu = torch.einsum('na,na->n', z, z)
    return torch.sum(mu+tr-ld)

ce = nn.CrossEntropyLoss()

# snpvae = SNPVAE(10433, 30)
snpvae = SNPVAE(35621, 100)
optim = torch.optim.Adam(snpvae.parameters(), lr=1e-3, weight_decay=0)

xclass = torch.from_numpy(snps).long().cuda()
x = torch.cat([xclass == 0, xclass == 1, xclass == 2], dim=1).float().cuda()
sex_t = torch.from_numpy(sex).float().cuda()
race_t = torch.from_numpy(race).float().cuda()

nepochs = 2000
pperiod = 50

for e in range(nepochs):
    optim.zero_grad()
    z = snpvae.enc(x[:800])
    logits = snpvae.dec(z, sex_t[:800], race_t[:800])
    loss_ce = ce(logits, xclass[:800])
    loss_kl = klloss2(z)
    # loss_C, loss_mu, C, _ = latent_loss(z)
    loss_decor = decor_loss(z, sex_t[:800], race_t[:800])
    # Generative loss
    sx_t = random.randint(0,1)*torch.ones(100).long().cuda()
    rc_t = random.randint(0,1)*torch.ones(100).long().cuda()
    z = snpvae.gen(100)
    logits = snpvae.dec(z, sx_t, rc_t)
    logits = torch.cat([logits[:,0], logits[:,1], logits[:,2]], dim=1).float().cuda()
    sx_hat = logits@sex_w + sex_i
    rc_hat = logits@race_w + race_i
    sex_loss = ce(torch.stack([-sx_hat, sx_hat], dim=1), sx_t)
    race_loss = ce(torch.stack([-rc_hat, rc_hat], dim=1), rc_t)
    # (20*loss_ce+1e-4*loss_C+1e-4*loss_mu+1e-2*loss_decor+0.01*sex_loss+0.01*race_loss).backward()
    (loss_ce+2e-4*loss_kl+2e-4*loss_decor+0.001*sex_loss+0.001*race_loss).backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        # print(f'{e} {pretty(loss_ce)} {pretty(loss_C)} {pretty(loss_mu)} {pretty(loss_decor)} {pretty(sex_loss)} {pretty(race_loss)}')
        print(f'{e} {pretty(loss_ce)} {pretty(loss_kl)} {pretty(loss_decor)} {pretty(sex_loss)} {pretty(race_loss)}')

print('Done')

tensor([-1958.0067, -1984.7115, -1968.2627, -1962.3083,        nan,        nan,
        -1926.1584,        nan, -1991.4304,        nan,        nan, -2000.2439,
        -1949.2017, -1985.9413, -1973.5046, -1974.4150, -1974.7781,        nan,
        -1992.0706, -2023.5323,        nan, -1999.6013,        nan, -2000.4510,
               nan, -1955.8319,        nan, -1981.3052, -1940.6836,        nan,
        -1985.1854, -1981.6063, -1973.7458,        nan,        nan, -1956.8263,
               nan, -1965.9131,        nan, -1990.7683, -2001.2771,        nan,
               nan,        nan, -1979.7769, -2000.0646, -1981.8816, -1971.1108,
               nan,        nan,        nan,        nan, -1973.8137,        nan,
        -1967.8628,        nan, -1981.9492, -1966.6049, -1982.8804, -1975.8392,
               nan,        nan, -1995.8494, -2002.4097, -1977.5991,        nan,
        -1978.1053, -1976.7570,        nan,        nan,        nan,        nan,
               nan, -1965.1519,        n

KeyboardInterrupt: 

In [64]:
# Save VAE

torch.save(snpvae.state_dict(), '/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_snps_big_1000_z100_cov2.torch')

print('Done')

Done


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

class SNPVAE(nn.Module):
    def __init__(self, snpd, ld):
        super(SNPVAE, self).__init__()
        self.ld = ld
        self.enc1 = nn.Linear(snpd*3, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+2, 1000).float().cuda()
        self.dec20 = nn.Linear(1000, snpd).float().cuda()
        self.dec21 = nn.Linear(1000, snpd).float().cuda()
        self.dec22 = nn.Linear(1000, snpd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)

    def dec(self, z, sex, race):
        z = torch.cat([z, sex.unsqueeze(1), race.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x0 = self.dec20(x)
        x1 = self.dec21(x)
        x2 = self.dec22(x)
        return torch.stack([x0, x1, x2], dim=1)

def latent_loss(z):
    C = z.T@z
    mu = torch.mean(z, dim=0)
    tgt1 = torch.eye(z.shape[-1]).float().cuda()*len(z)/10
    tgt2 = torch.zeros(z.shape[-1]).float().cuda()
    lossC = rmse(C, tgt1)
    lossmu = rmse(mu, tgt2)
    return lossC, lossmu, C, mu

def decor_loss(z, sex, race):
    sex = sex - torch.mean(sex)
    race = race - torch.mean(race)
    sexp = torch.einsum('n,nz->z', sex, z)
    racep = torch.einsum('n,nz->z', race, z)
    tgt = torch.zeros(z.shape[-1]).float().cuda()
    loss_sex = rmse(sexp, tgt)
    loss_race = rmse(racep, tgt)
    return loss_sex + loss_race

ce = nn.CrossEntropyLoss()

# snpvae = SNPVAE(10433, 30)
snpvae = SNPVAE(35621, 100)
snpvae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_snps_big_1000_z100_cov2.torch'))
snpvae.eval()

xclass = torch.from_numpy(snps).long().cuda()
x = torch.cat([xclass == 0, xclass == 1, xclass == 2], dim=1).float().cuda()
sex_t = torch.from_numpy(sex).float().cuda()
race_t = torch.from_numpy(race).float().cuda()

print('Done')

Done


In [5]:
with torch.no_grad():
    z = snpvae.enc(x)
    logits = snpvae.dec(z, sex_t, race_t)
    xhat = torch.argmax(logits, dim=1)

for i in range(800,1000):
    print(torch.sum(xhat[i] == xclass[i]))
print(xhat[2])
print(xclass[2])

tensor(33629, device='cuda:0')
tensor(29759, device='cuda:0')
tensor(35621, device='cuda:0')
tensor(33496, device='cuda:0')
tensor(35327, device='cuda:0')
tensor(35504, device='cuda:0')
tensor(35431, device='cuda:0')
tensor(34295, device='cuda:0')
tensor(33910, device='cuda:0')
tensor(35308, device='cuda:0')
tensor(29521, device='cuda:0')
tensor(35581, device='cuda:0')
tensor(35425, device='cuda:0')
tensor(34509, device='cuda:0')
tensor(32095, device='cuda:0')
tensor(32433, device='cuda:0')
tensor(34485, device='cuda:0')
tensor(35492, device='cuda:0')
tensor(35406, device='cuda:0')
tensor(35494, device='cuda:0')
tensor(35621, device='cuda:0')
tensor(35478, device='cuda:0')
tensor(33117, device='cuda:0')
tensor(35364, device='cuda:0')
tensor(32256, device='cuda:0')
tensor(31379, device='cuda:0')
tensor(31255, device='cuda:0')
tensor(32366, device='cuda:0')
tensor(30948, device='cuda:0')
tensor(31583, device='cuda:0')
tensor(34977, device='cuda:0')
tensor(35475, device='cuda:0')
tensor(3

In [6]:
class VAE(nn.Module):
    def __init__(self, fcd, ld):
        super(VAE, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+6, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, fcd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, rest, nback, emoid):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), 
                       rest.unsqueeze(1), nback.unsqueeze(1), emoid.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        return x

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

vae = VAE(34716, 30)
vae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_1000_z30_cov6.torch'))
vae.eval()

print('Done')

Done


In [10]:
class Similarity(nn.Module):
    def __init__(self, ldfc, ldsnp):
        super(Similarity, self).__init__()
        self.fca1 = nn.Linear(ldfc, 20).float().cuda()
        self.fcb1 = nn.Linear(ldsnp, 20).float().cuda()
        self.fc2 = nn.Linear(40, 1).float().cuda()

    def forward(self, zfc, zsnp):
        zfc = F.relu(self.fca1(zfc))
        zsnp = F.relu(self.fcb1(zsnp))
        z = torch.cat([zfc, zsnp], dim=1)
        z = self.fc2(z)
        return z

def cl_loss(sims, pidcs, tau):
    pp = sims[pidcs]
    ep = torch.sum(torch.exp(pp/tau))
    en = torch.sum(torch.exp(sims/tau))
    loss = -torch.log(ep/en)
    return loss

sim = Similarity(30, 50)
optim = torch.optim.Adam(sim.parameters(), lr=1e-4, weight_decay=1e-6)

with torch.no_grad():
    zrest = vae.enc(torch.from_numpy(rest).float().cuda())
    znback = vae.enc(torch.from_numpy(nback).float().cuda())
    zemoid = vae.enc(torch.from_numpy(emoid).float().cuda())
    zsnp = snpvae.enc(x)

nepochs = 20000
pperiod = 500
nb = 200

for e in range(nepochs):
    optim.zero_grad()
    sidcs = np.random.permutation(len(x))[:nb]
    idcs1 = []
    idcs2 = []
    pidcs = []
    for i in range(nb):
        idcs1.append(np.ones(nb)*sidcs[i])
        idcs2.append(sidcs)
        pidcs.append(nb*i+i)
    idcs1 = np.array(np.concatenate(idcs1))
    idcs2 = np.array(np.concatenate(idcs2))
    pidcs = np.array(pidcs)
    if e % 3 == 0:
        ztask = zrest
    elif e % 3 == 1:
        ztask = znback
    else:
        ztask = zemoid
    sims = sim(ztask[idcs1], zsnp[idcs2])
    loss_cl = cl_loss(sims, pidcs, 1)
    loss_cl.backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(loss_cl)}')

print('Complete')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (40000x100 and 50x20)

In [31]:
# Test PC imputation

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, FactorAnalysis

xreal = x.detach().cpu().numpy()

xtr = xreal[:800]
xt = xreal[800:]
sx = torch.from_numpy(sex[800:]).float().cuda()
rc = torch.from_numpy(race[800:]).float().cuda()
x2t = snps[800:]

accs_pca = []
accs_vae = []
# accs_sample = []
n = 35000

for _ in range(20):

    # xtr, xt, x2tr, x2t, _, zr, _, zn, _, ze, _, sx, _, rc = train_test_split(xreal, snps, zrest, znback, zemoid, sex_t, race_t, train_size=0.8)

    # sx[:] = torch.randint(low=0,high=2,size=[len(sx)])
    # rc[:] = torch.randint(low=0,high=2,size=[len(rc)])
    
    miss = np.random.permutation(snps.shape[-1])[:n]
    # pca = PCA(n_components=1).fit(xtr)
    
    xtt = xt+0
    cls = np.random.randint(low=0, high=3, size=(n))
    xtt[:,miss] = (cls == 0)+0
    xtt[:,miss+1*snps.shape[-1]] = (cls == 1)+0
    xtt[:,miss+2*snps.shape[-1]] = (cls == 2)+0
    
    # xtt_pca = pca.transform(xtt)
    # xtt_bak = pca.inverse_transform(xtt_pca)
    
    # xtt_bak = xtt_bak.reshape(xtt_bak.shape[0], 3, snps.shape[-1])
    # xtt_bak = np.argmax(xtt_bak, axis=1)
    
    # acc = np.mean(np.sum(xtt_bak[:,miss] == x2t[:,miss], axis=1))
    # accs_pca.append(acc/n)
    accs_pca.append(1)
    
    with torch.no_grad():
        z = snpvae.enc(torch.from_numpy(xtt).float().cuda())
        logits = snpvae.dec(z, sx, rc)
        logits = torch.argmax(logits, dim=1).detach().cpu().numpy()
    
    acc = np.mean(np.sum(logits[:,miss] == x2t[:,miss], axis=1))
    accs_vae.append(acc/n)

    print(accs_pca[-1], accs_vae[-1])

    # myaccs = []
    
    # for i in range(len(zr)):
    #     with torch.no_grad():
    #         zz = snpvae.gen(100)
    #         sims1 = sim(zr[i:i+1].repeat([100,1]), zz)
    #         sims2 = sim(zn[i:i+1].repeat([100,1]), zz)
    #         sims3 = sim(ze[i:i+1].repeat([100,1]), zz)
    #         sims = sims1+sims2+sims3

    #         idcs = torch.argsort(sims, descending=True)
    #         best = 0
    #         for j in idcs[:50]:
    #             logits = snpvae.dec(zz[j:j+1], sx[i:i+1], rc[i:i+1])
    #             logits = torch.argmax(logits, dim=1).detach().cpu().numpy()
    #             acc = np.sum(logits[0,miss] == x2t[i,miss])
    #             if acc > best:
    #                 best = acc
    #         myaccs.append(best/n)

    # print(np.mean(myaccs))

print('---')
print(np.mean(accs_pca))
print(np.std(accs_pca))
print(np.mean(accs_vae))
print(np.std(accs_vae))

1 0.5060987525150905
1 0.5055681287726358
1 0.5063547686116701
1 0.506019879275654
1 0.5075904225352114
1 0.5070614084507042
1 0.5079449496981892
1 0.5068706639839033
1 0.506313722334004
1 0.5060567404426559
1 0.5034149698189134
1 0.5059128370221327
1 0.5074739637826963
1 0.5060327565392354
1 0.5091603219315896
1 0.5066175452716298
1 0.5060531187122737
1 0.5069287726358149
1 0.5048663179074446
1 0.505982615694165
---
1.0
0.0
0.5064161327967807
0.0011519840217475612
